In [ ]:
!pip install -q transformers datasets accelerate sentencepiece



In [ ]:
from google.colab import drive
drive.mount('/content/drive')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/medical-t5-data/medical_dialogue.csv")
df.head()


,input,output
0,doctor i have been experiencing sudden and fre...,well based on what youre telling me it sounds ...
1,doctor i have been experiencing a hoarse voice...,its possible that you have a vocal cord polyp ...
2,doctor ive been experiencing a condition calle...,you will need to undergo a complete physical s...
3,doctor i think ive been poisoned i drank some ...,im sorry to hear that we need to run a few tes...
4,doctor ive been having discomfort and dryness ...,based on your symptoms it sounds like you may ...


In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

len(train_df), len(test_df)


(4837, 538)

In [ ]:
from datasets import Dataset

train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
test_ds = Dataset.from_pandas(test_df.reset_index(drop=True))


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


In [ ]:
max_input_length = 256
max_output_length = 256

def preprocess_function(batch):
    inputs = ["symptom: " + x for x in batch["input"]]

    model_inputs = tokenizer(
        inputs,
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch["output"],
            padding="max_length",
            truncation=True,
            max_length=max_output_length,
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_tokenized = train_ds.map(preprocess_function, batched=True)
test_tokenized = test_ds.map(preprocess_function, batched=True)


Map:   0%|          | 0/4837 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4118: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/538 [00:00<?, ? examples/s]

In [ ]:
!pip install -q -U transformers datasets accelerate sentencepiece


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="/content/med-t5",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=5,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer.train()


Step,Training Loss
5,0.629600
10,0.829100
15,0.657400
20,0.523300
25,0.546100
30,0.516000
35,0.695000
40,0.952200
45,0.729900
50,0.496500


TrainOutput(global_step=7257, training_loss=0.534221411391988, metrics={'train_runtime': 839.3596, 'train_samples_per_second': 17.288, 'train_steps_per_second': 8.646, 'total_flos': 982513807785984.0, 'train_loss': 0.534221411391988, 'epoch': 3.0})

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
def generate_answer(text):
    input_text = "symptom: " + text
    inputs = tokenizer.encode(input_text, return_tensors="pt").to(model.device)

    outputs = model.generate(
        inputs,
        max_length=256,
        num_beams=4,
        early_stopping=True,
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

sample_text = df["input"].iloc[0]
print("INPUT:")
print(sample_text)

print("\nMODEL OUTPUT:")
print(generate_answer(sample_text))

print("\nREAL OUTPUT:")
print(df["output"].iloc[0])


INPUT:
doctor i have been experiencing sudden and frequent panic attacks i dont know what to do

MODEL OUTPUT:
based on your symptoms it seems like you may be experiencing panic attacks we will need to run some tests to confirm the diagnosis

REAL OUTPUT:
well based on what youre telling me it sounds like you may be suffering from panic disorder the best course of action is to start with psychotherapy and mental health counseling additionally we should conduct an electrocardiogram to make sure that there are no physical issues causing your panic attacks we will also need to perform a depression screen and a toxicology screen to rule out any other underlying causes finally i would recommend a comprehensive psychological and psychiatric evaluation and therapy to help manage your symptoms


In [ ]:
model.save_pretrained("/content/med-t5-trained")
tokenizer.save_pretrained("/content/med-t5-trained")


('/content/med-t5-trained/tokenizer_config.json',
 '/content/med-t5-trained/special_tokens_map.json',
 '/content/med-t5-trained/spiece.model',
 '/content/med-t5-trained/added_tokens.json')

In [ ]:
trainer.evaluate()


{'eval_loss': 0.5031563639640808,
 'eval_runtime': 8.3912,
 'eval_samples_per_second': 64.115,
 'eval_steps_per_second': 32.057,
 'epoch': 3.0}

In [ ]:
model_dir = "/content/drive/MyDrive/Colab Notebooks/medical-t5-data/med-t5-model"

model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)


('/content/drive/MyDrive/Colab Notebooks/medical-t5-data/med-t5-model/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/medical-t5-data/med-t5-model/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/medical-t5-data/med-t5-model/spiece.model',
 '/content/drive/MyDrive/Colab Notebooks/medical-t5-data/med-t5-model/added_tokens.json')

In [ ]:
def generate_answer(text):
    input_text = "symptom: " + text

    inputs = tokenizer.encode(input_text, return_tensors="pt").to(model.device)

    outputs = model.generate(
        inputs,
        max_length=256,
        num_beams=4,
        early_stopping=True,
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
sample_text = df["input"].iloc[0]

print("INPUT:")
print(sample_text)

print("\nMODEL OUTPUT:")
print(generate_answer(sample_text))

print("\nREAL OUTPUT:")
print(df["output"].iloc[0])


INPUT:
doctor i have been experiencing sudden and frequent panic attacks i dont know what to do

MODEL OUTPUT:
based on your symptoms it seems like you may be experiencing panic attacks we will need to run some tests to confirm the diagnosis

REAL OUTPUT:
well based on what youre telling me it sounds like you may be suffering from panic disorder the best course of action is to start with psychotherapy and mental health counseling additionally we should conduct an electrocardiogram to make sure that there are no physical issues causing your panic attacks we will also need to perform a depression screen and a toxicology screen to rule out any other underlying causes finally i would recommend a comprehensive psychological and psychiatric evaluation and therapy to help manage your symptoms


In [ ]:
new_symptom = "doctor i have been feeling very tired, gaining weight, and feel thirsty all the time what could be wrong"
print(generate_answer(new_symptom))


based on your symptoms it seems like you may be suffering from a condition called syphilis
